In [1]:
import pandas as pd
import numpy as np
import torch

from transformers import AutoModel, AutoTokenizer 

import time

import sys

import re

from tqdm import tqdm

import faiss

from transformers import AutoTokenizer, AutoModel
from datasets import load_dataset

from pprint import pprint
import io

import logging
logging.basicConfig(level=logging.INFO)

import matplotlib.pyplot as plt

from helper import stream

import psycopg2

len_dataset = 2326839

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

c:\Users\faree\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda', index=0)

In [2]:
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-medium")
model = AutoModel.from_pretrained('prajjwal1/bert-medium', output_hidden_states=True)
model.to(device)

c:\Users\faree\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\faree\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of the model checkpoint at prajjwal1/bert-medium were not used when initializing BertModel: ['cls.predicti

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 512, padding_idx=0)
    (position_embeddings): Embedding(512, 512)
    (token_type_embeddings): Embedding(2, 512)
    (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-7): 8 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=512, out_features=512, bias=True)
            (key): Linear(in_features=512, out_features=512, bias=True)
            (value): Linear(in_features=512, out_features=512, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=512, out_features=512, bias=True)
            (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
    

In [3]:
dataset = load_dataset('json', data_files='dataset/arxiv_data.json', split='train', streaming=True)

In [4]:
dataset

In [5]:
next(iter(dataset))

{'id': '0704.0001',
 'submitter': 'Pavel Nadolsky',
 'authors': "C. Bal\\'azs, E. L. Berger, P. M. Nadolsky, C.-P. Yuan",
 'title': 'Calculation of prompt diphoton production cross sections at Tevatron and\n  LHC energies',
 'comments': '37 pages, 15 figures; published version',
 'journal-ref': 'Phys.Rev.D76:013009,2007',
 'doi': '10.1103/PhysRevD.76.013009',
 'report-no': 'ANL-HEP-PR-07-12',
 'categories': 'hep-ph',
 'license': None,
 'abstract': '  A fully differential calculation in perturbative quantum chromodynamics is\npresented for the production of massive photon pairs at hadron colliders. All\nnext-to-leading order perturbative contributions from quark-antiquark,\ngluon-(anti)quark, and gluon-gluon subprocesses are included, as well as\nall-orders resummation of initial-state gluon radiation valid at\nnext-to-next-to-leading logarithmic accuracy. The region of phase space is\nspecified in which the calculation is most reliable. Good agreement is\ndemonstrated with data from th

In [6]:
def tokenize_dataset(data):
    return tokenizer(data['abstract'], padding=True, truncation=True, max_length=256, return_tensors='pt')

tokenized_dataset = dataset.map(tokenize_dataset, batched=True, batch_size=512, remove_columns=['id', 'submitter', 'authors', 'title', 'comments', 'journal-ref', 'doi', 'report-no', 'categories', 'license', 'versions', 'update_date', 'authors_parsed'])

In [7]:
dataloader = torch.utils.data.DataLoader(tokenized_dataset, batch_size=128)

In [8]:
print(len(next(iter(dataloader))['abstract']))

128


In [9]:
def save_to_disk(data, filename):
    np.savez(filename, data)

In [10]:
%%time
model.eval()
embeddings = []
i = 0
with torch.no_grad():
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        
        start = time.time()
        
        outputs = model(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.hidden_states
        embeddings.append(hidden_states[-2].mean(dim=1).cpu().numpy()) # average the 256 vectors
        end = time.time()
        print(f'Batch of size {input_ids.shape[0]} took {end - start:.2f} seconds\tBatch: {i}/{int(np.ceil(len_dataset/128))}\tEmbedding Shape: {embeddings[-1].shape}')
        
        i+= 1
        
        if i % 1000 == 0:
            end = time.time()
            embeddings = np.array(embeddings)
            save_to_disk(embeddings, f'bert_medium_embeddings/embeddings_{i}.npz')
            embeddings = embeddings.tolist()
            embeddings = []
            
if len(embeddings) > 0:
    embeddings = np.array(embeddings)
    save_to_disk(embeddings, f'bert_medium_embeddings/embeddings_{i}.npz')

Batch of size 128 took 3.58 seconds	Batch: 0/18179	Embedding Shape: (128, 512)
Batch of size 128 took 0.18 seconds	Batch: 1/18179	Embedding Shape: (128, 512)
Batch of size 128 took 0.17 seconds	Batch: 2/18179	Embedding Shape: (128, 512)
Batch of size 128 took 0.17 seconds	Batch: 3/18179	Embedding Shape: (128, 512)
Batch of size 128 took 0.17 seconds	Batch: 4/18179	Embedding Shape: (128, 512)
Batch of size 128 took 0.17 seconds	Batch: 5/18179	Embedding Shape: (128, 512)
Batch of size 128 took 0.17 seconds	Batch: 6/18179	Embedding Shape: (128, 512)
Batch of size 128 took 0.17 seconds	Batch: 7/18179	Embedding Shape: (128, 512)
Batch of size 128 took 0.17 seconds	Batch: 8/18179	Embedding Shape: (128, 512)
Batch of size 128 took 0.17 seconds	Batch: 9/18179	Embedding Shape: (128, 512)
Batch of size 128 took 0.17 seconds	Batch: 10/18179	Embedding Shape: (128, 512)
Batch of size 128 took 0.17 seconds	Batch: 11/18179	Embedding Shape: (128, 512)
Batch of size 128 took 0.17 seconds	Batch: 12/1817

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (179,) + inhomogeneous part.

In [25]:
# load embeddings
embeddings = np.load('bert_tiny_embeddings/embeddings_1000.npz')['arr_0']

In [26]:
embeddings.shape

(1000, 128, 128)

In [27]:
batch_1 = embeddings[0]

In [28]:
batch_1.shape

(128, 128)

In [29]:
batch_1[0][0]

-0.20750296

In [30]:
embeddings[0][0][1] == batch_1[0][1]

True

In [16]:
embeddings_copy = embeddings.copy()

In [37]:
# CAUTIONl THIS IS ON T5 MODE | MAKE SURE TO DOUBLE CHECK FOR WHICH MODEL YOU WANT

def vec_to_sql_string(vector):
    return str(vector.tolist())

def from_sql_to_list(string):
    sql_string = re.findall(r'\[.*?\]', string)
    lst = [eval(i) for i in sql_string]
    return torch.tensor(np.array(lst), device=device, dtype=torch.float64)

def connect_to_db():
    conn = psycopg2.connect(
        host="localhost",
        port=5432,
        database="vector_database",
        user="postgres",
        password="admin"
    )
    conn.autocommit = True

    cur = conn.cursor()
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector")
    return cur, conn

cur, conn = connect_to_db()
def save_batch(batch): 
    i = 0
    for embedding in tqdm(batch):   
        sql_string = vec_to_sql_string(batch[i])
        cur.execute("INSERT INTO bert_tiny_article_embeddings VALUES (%s, %s)", (i, sql_string))
        i += 1

In [38]:
cur.execute("drop table bert_tiny_article_embeddings;")
cur.execute("create table bert_tiny_article_embeddings(article_ID int primary key, embedding vector(128));")
conn.commit()

In [39]:
embeddings = embeddings.reshape(-1, 128)
embeddings.shape

(128000, 128)

In [32]:
print(embeddings[0])
embeddings[0].shape

[-2.07502961e-01  2.59919941e-01  7.66457617e-02 -5.33710599e-01
  5.50580859e-01 -8.13874751e-02 -3.62138271e-01 -9.54170942e-01
 -1.51484156e+00  1.13544524e+00 -3.59661616e-02  1.58597350e-01
  4.75086540e-01 -3.66304457e-01  2.18662202e-01 -4.31249857e-01
 -7.63916433e-01 -4.25528944e-01 -3.63185614e-01  2.66734034e-01
  5.95240951e-01 -9.55599070e-01  4.20162044e-02  1.26973748e+00
  2.09576225e+00  2.87510306e-01  2.61032343e-01  6.92113578e-01
  1.09987879e+00  1.71180397e-01 -7.44505525e-01 -9.49997365e-01
  5.68992972e-01 -1.36842501e+00 -2.43074775e-01 -6.09823108e-01
  3.82721484e-01 -6.26649857e-01  9.58747789e-02 -4.22826350e-01
  4.20409858e-01  6.02428168e-02  4.56818342e-01 -1.71236944e+00
  5.75114310e-01 -3.54278758e-02 -4.64557469e-01  6.18991375e-01
  2.36144662e-01 -3.37534070e-01  1.25275888e-02  2.26042604e+00
  2.75421500e-01 -8.41275096e-01  3.02151859e-01 -1.97863936e-01
 -2.63559967e-01  1.00775838e-01  1.00573704e-01  1.27844644e+00
  1.08576727e+00 -5.45933

(128,)

In [60]:
t = vec_to_sql_string(embeddings[0])
t
# get the first 100 embeddings
batch = embeddings[:100]

In [31]:
%%time
save_batch(batch)

100%|██████████| 100/100 [00:04<00:00, 21.55it/s]

CPU times: total: 0 ns
Wall time: 4.64 s


In [ ]:
# checking values against the database to make sure they are the same
batch[4].tolist()

In [68]:
%%time
# insert to faiss
index = faiss.IndexFlatL2(128)
index.add(embeddings)
print(index.ntotal)

128000
CPU times: total: 109 ms
Wall time: 316 ms


In [69]:
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x0000029584C3CB40> >

In [70]:
# find the 4 nearest neighbors of the first vector in embeddings
D, I = index.search(embeddings[:1], 4)

In [71]:
D.shape

(1, 4)

In [72]:
I.shape

(1, 4)

In [73]:
# explain what D and I are
print(D)
print(I)

# get the vectors for the first 4 nearest neighbors
embeddings[I[0]]


[[0.       8.651371 9.205639 9.390191]]
[[    0 31696 84121 47304]]


array([[-0.4060021 ,  0.02999023,  0.34609616, ..., -0.04021358,
         0.19924636,  0.20150217],
       [-0.54234874,  0.20500174,  0.52792776, ..., -0.06560621,
         0.11267843,  0.2924931 ],
       [-0.40299344,  0.11314876,  0.342015  , ..., -0.03731213,
         0.23403625,  0.05377698],
       [-0.33520997,  0.07710867,  0.40404385, ..., -0.06937364,
         0.06732745,  0.20221093]], dtype=float32)

In [15]:
def load_all_embeddings():
    e_list = []
    for i in range(1, 19):
        embeddings = np.load(f'bert_medium_embeddings/embeddings_{i}000.npz')['arr_0'].reshape(-1, 512)
        e_list.append(embeddings)
    
    e = np.concatenate(e_list, axis=0)
    return e

embeddings_array = load_all_embeddings()



In [16]:
embeddings_array.shape

(2304000, 512)

In [40]:
save_batch(embeddings_array)

100%|██████████| 2304000/2304000 [41:43<00:00, 920.45it/s] 


In [17]:
%%time
index = faiss.IndexFlatL2(512)
index.add(embeddings_array)
print(index.ntotal)

2304000
CPU times: total: 93.8 ms
Wall time: 1.14 s


In [18]:
# pickle the index and save it to disk
faiss.write_index(index, 'Indexes/bert_medium_embeddings.index')

In [17]:
loaded_index = faiss.read_index('Indexes/bert_tiny_embeddings.index')

In [18]:
# Sanity check to make sure the index is the same (could do a better check but this is fine for now)
assert loaded_index.ntotal == 2304000

In [19]:
D, I = loaded_index.search(embeddings_array[:1], 4)

In [20]:
pprint(D)
pprint(I)

embeddings_array[I[0]]

array([[0.       , 2.5346384, 2.707406 , 2.748757 ]], dtype=float32)
array([[      0,  112449, 2159515,  320436]], dtype=int64)


array([[-2.07502961e-01,  2.59919941e-01,  7.66457617e-02,
        -5.33710599e-01,  5.50580859e-01, -8.13874751e-02,
        -3.62138271e-01, -9.54170942e-01, -1.51484156e+00,
         1.13544524e+00, -3.59661616e-02,  1.58597350e-01,
         4.75086540e-01, -3.66304457e-01,  2.18662202e-01,
        -4.31249857e-01, -7.63916433e-01, -4.25528944e-01,
        -3.63185614e-01,  2.66734034e-01,  5.95240951e-01,
        -9.55599070e-01,  4.20162044e-02,  1.26973748e+00,
         2.09576225e+00,  2.87510306e-01,  2.61032343e-01,
         6.92113578e-01,  1.09987879e+00,  1.71180397e-01,
        -7.44505525e-01, -9.49997365e-01,  5.68992972e-01,
        -1.36842501e+00, -2.43074775e-01, -6.09823108e-01,
         3.82721484e-01, -6.26649857e-01,  9.58747789e-02,
        -4.22826350e-01,  4.20409858e-01,  6.02428168e-02,
         4.56818342e-01, -1.71236944e+00,  5.75114310e-01,
        -3.54278758e-02, -4.64557469e-01,  6.18991375e-01,
         2.36144662e-01, -3.37534070e-01,  1.25275888e-0